In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sunpy
import sunpy.map
from sunpy.coordinates import propagate_with_solar_surface
import astropy
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
import astropy.constants as const
from astropy.io import fits
from astropy.time import Time
from astropy.visualization import (ImageNormalize, AsinhStretch)
import pyvista as pv
import os

In [3]:
hri_174_map_shifted_toaia = sunpy.map.Map("../../src/coalign_map/20221024/hri_174_map_shifted_toaia.fits")
spice_NeVIII_intmap = sunpy.map.Map("../../src/SPICE/20221024/lvl3/NeVIII_int_deconv_plowman2023.fits")
spice_observer = spice_NeVIII_intmap.observer_coordinate

In [4]:
hri_174_map_shifted_toaia_all_coord = sunpy.map.all_coordinates_from_map(hri_174_map_shifted_toaia)
hri_174_map_shifted_toaia_all_coord = hri_174_map_shifted_toaia_all_coord.make_3d()

In [5]:
hri_174_map_shifted_toaia_all_coord_hee = hri_174_map_shifted_toaia_all_coord.transform_to(hri_174_map_shifted_toaia.observer_coordinate)

In [6]:
seed_pixel_coord0_spice_frame = SkyCoord(-29.28289379*u.deg, 15.27802344*u.deg, 6.957e+08*u.m, frame="heliographic_stonyhurst",
                                          obstime=spice_NeVIII_intmap.date)

In [7]:
seed_pixel_coord0_spice_frame.cartesian.xyz.to_value("AU")

array([ 0.00391286, -0.00219426,  0.00122541])

In [8]:
with propagate_with_solar_surface(rotation_model="rigid"):
    hri_174_map_shifted_toaia_all_coord_hee_spice = hri_174_map_shifted_toaia_all_coord_hee.transform_to(spice_observer.frame)

In [9]:
hri_174_map_shifted_toaia_3d_grid = pv.StructuredGrid(hri_174_map_shifted_toaia_all_coord_hee_spice.cartesian.x.to_value("AU"),
                                                      hri_174_map_shifted_toaia_all_coord_hee_spice.cartesian.y.to_value("AU"),
                                                     hri_174_map_shifted_toaia_all_coord_hee_spice.cartesian.z.to_value("AU"))

In [10]:
hri_174_map_shifted_toaia_all_coord_hee_spice[0,0]

<HeliographicStonyhurst Coordinate (obstime=2022-10-24T23:15:35.745, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, m)
    (-37.57874364, 8.80504748, 6.957e+08)>

In [11]:
hri_174_map_shifted_toaia_all_coord_hee_spice[0,0]

<HeliographicStonyhurst Coordinate (obstime=2022-10-24T23:15:35.745, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, m)
    (-37.57874364, 8.80504748, 6.957e+08)>

In [12]:
vec_ep = np.array([ 31.02313114, -19.57743934,   4.15088941])
hinode_unit_vec = np.array([0.99615634, 0.00444352, 0.0874803 ])
hinode_vel_mag = np.abs(-31.180016963686196)
hinode_vel_vec = hinode_unit_vec*hinode_vel_mag
spice_unit_vec = np.array([ 0.62767719, -0.77050905,  0.11107271])
spice_vel_mag = np.abs(-35.01815653373054)
spice_unit_vec = spice_unit_vec*spice_vel_mag

In [46]:
pv_sphere = pv.Sphere(radius=seed_pixel_coord0_spice_frame.radius.to_value("AU")*0.99,
                      theta_resolution=50, phi_resolution=50)

pv_meridians = []
for ii in range(12):
    phi_ = np.deg2rad(15.*ii)
    norm_vec_ = np.array([np.cos(phi_), np.sin(phi_), 0.])
    pv_meridians.append(pv.CircularArcFromNormal(center=[0,0,0],
                                                 resolution=300,
                                                 normal=norm_vec_,
                                                 polar=[0,0,seed_pixel_coord0_spice_frame.radius.to_value("AU")*1.01],
                                                 angle=360))
    
pv_latitudes = []
for ii in range(11):
    theta_ = np.deg2rad(15.*(ii+1))
    lat_ = np.pi/2 - theta_
    phis = np.linspace(0,2*np.pi,300)
    latitude_x = seed_pixel_coord0_spice_frame.radius.to_value("AU")*np.cos(lat_)*np.cos(phis)*1.01
    latitude_y = seed_pixel_coord0_spice_frame.radius.to_value("AU")*np.cos(lat_)*np.sin(phis)*1.01
    latitude_z = seed_pixel_coord0_spice_frame.radius.to_value("AU")*np.sin(lat_)*1.01*np.ones_like(phis)   
    latitude_points = np.column_stack((latitude_x, latitude_y, latitude_z))
    pv_latitudes.append(pv.lines_from_points(latitude_points))


In [47]:
pv_plotter = pv.Plotter(notebook=True)

hri_174_map_shifted_toaia_3d_norm = ImageNormalize(vmin=0,vmax=1.3e4,stretch=AsinhStretch(0.03))
pv_plotter.add_mesh(hri_174_map_shifted_toaia_3d_grid,scalars=hri_174_map_shifted_toaia_3d_norm(hri_174_map_shifted_toaia.data), cmap="solar orbiterhri_euv174")
pv_plotter.add_mesh(pv_sphere, show_edges=False, opacity=1, color="black", lighting=False, label="Test Mesh")
pv_plotter.add_arrows(seed_pixel_coord0_spice_frame.cartesian.xyz.to_value("AU"),vec_ep, mag=1e-5,color="lime")
pv_plotter.add_arrows(seed_pixel_coord0_spice_frame.cartesian.xyz.to_value("AU"),hinode_vel_vec, mag=1e-5,color="red")
pv_plotter.add_arrows(seed_pixel_coord0_spice_frame.cartesian.xyz.to_value("AU"),spice_unit_vec, mag=1e-5,color="blue")

for pv_meridian_ in pv_meridians:
    pv_plotter.add_mesh(pv_meridian_, color="white", line_width=1,opacity=0.4)
for pv_latitude_ in pv_latitudes:
    pv_plotter.add_mesh(pv_latitude_, color="white", line_width=1,opacity=0.4)

pv_plotter.add_axes(line_width=5, labels_off=False)

if os.path.exists("../../figs/test_figs/hri_174_map_shifted_toaia_3d.html"):
    os.remove("../../figs/test_figs/hri_174_map_shifted_toaia_3d.html")
pv_plotter.export_html('../../figs/test_figs/hri_174_map_shifted_toaia_3d.html')  
# pv_plotter.export_vtksz("../../figs/test_figs/hri_174_map_shifted_toaia_3d.vtksz")

# del pv_plotter
